In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-10-26 23:27:04.604905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def add_location(frame, location):
    frame["location"] = location
    return frame

In [3]:
def removeNaN(frame):
    frame = frame.fillna(method="ffill")
    frame = frame.fillna(method="bfill")
    frame = frame.fillna(value=0)
    return frame

In [7]:
def averageFeatures(frame):
    frame["date_forecast"] = frame["date_forecast"].apply(lambda a : a.replace(minute=0))
    return frame.groupby(["date_forecast"])[frame.iloc[:,1:].columns].mean().reset_index()

Load and clean data

In [8]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')

train_b = pd.read_parquet('B/train_targets.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')

train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

train_a = add_location(train_a, "A")
train_b = add_location(train_b, "B")
train_c = add_location(train_c, "C")
X_train_estimated_a = X_train_estimated_a.iloc[:,1:]
X_train_estimated_b = X_train_estimated_b.iloc[:,1:]
X_train_estimated_c = X_train_estimated_c.iloc[:,1:]
X_train_estimated_a = add_location(averageFeatures(removeNaN(X_train_estimated_a)), "A")
X_train_estimated_b = add_location(averageFeatures(removeNaN(X_train_estimated_b)), "B")
X_train_estimated_c = add_location(averageFeatures(removeNaN(X_train_estimated_c)), "C")
X_train_observed_a = add_location(averageFeatures(removeNaN(X_train_observed_a)), "A")
X_train_observed_b = add_location(averageFeatures(removeNaN(X_train_observed_b)), "B")
X_train_observed_c = add_location(averageFeatures(removeNaN(X_train_observed_c)), "C")

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_65671/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_65671/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="bfill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_65671/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_65671/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = f

In [10]:
print(len(X_train_observed_a))
print(len(X_train_observed_b))
print(len(X_train_observed_c))
print(len(X_train_estimated_a))
print(len(X_train_estimated_b))
print(len(X_train_estimated_c))


targets = pd.concat([train_a, train_b, train_c])
train_features = pd.concat([
                     X_train_observed_a.iloc[:28000,:], 
                     X_train_observed_b.iloc[:28000,:], 
                     X_train_observed_c.iloc[:28000,:],
                     X_train_estimated_a.iloc[:4000,:],
                    X_train_estimated_b.iloc[:4000,:],
                    X_train_estimated_c.iloc[:4000,:]])
train_features = train_features.rename(columns={"date_forecast" : "time"})
trainset = pd.merge(train_features, targets, how="right", on=["time","location"]).dropna()

eval_features = pd.concat([X_train_observed_a.iloc[28000:,:], 
                     X_train_observed_b.iloc[28000:,:], 
                     X_train_observed_c.iloc[28000:,:],
                     X_train_estimated_a.iloc[4000:,:],
                    X_train_estimated_b.iloc[4000:,:],
                    X_train_estimated_c.iloc[4000:,:]])
eval_features = eval_features.rename(columns={"date_forecast" : "time"})

evalset = pd.merge(eval_features, targets, how="right", on=["time","location"]).dropna()
#trainset = trainset[trainset["sun_elevation:d"] >= 5]
#trainset.to_csv("trainset.csv")


trainsetX = trainset.iloc[:, 1:-2].reset_index().iloc[:,1:]
trainsetX = ((trainsetX-trainsetX.mean())/trainsetX.std()).fillna(value=0)
trainsetY = trainset.iloc[:, -1:].reset_index().iloc[:,1:]

evalsetX = evalset.iloc[:, 1:-2].reset_index().iloc[:, 1:]
evalsetX = ((evalsetX-evalsetX.mean())/evalsetX.std()).fillna(value=0)
evalsetY = evalset.iloc[:, -1:].reset_index().iloc[:, 1:]


display(train_features)



29668
29233
29207
4394
4394
4394


,time,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000e+00,0.000000,1728.949951,0.0,280.299988,0.000000,0.000000,0.000000,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.000,71.674995,1005.799988,250.0,0.0,0.0,0.0,0.175,348.036743,-3.774250,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.000,A
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000e+00,0.000000,1689.824951,0.0,280.299988,0.000000,0.000000,0.000000,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.025,68.000000,1004.650024,250.0,0.0,0.0,0.0,0.200,91.980751,-4.357250,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.000,A
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000e+00,0.000000,1563.224976,0.0,280.649994,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.100,67.949997,1003.950012,250.0,0.0,0.0,0.0,0.400,14.934750,-3.309500,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.000,A
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,2.086500e+02,0.750000,1283.425049,0.0,281.674988,0.300000,526.775024,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.0,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.125,73.875000,1003.449951,250.0,0.0,0.0,0.0,0.550,28.630251,-0.822500,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.000,A
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,3.246815e+04,23.100000,1003.500000,0.0,282.500000,11.975000,22068.949219,0.150000,282.975006,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.100,79.925003,1002.500000,250.0,0.0,0.0,0.0,0.250,41.997501,3.051250,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.000,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2023-04-14 09:00:00,4.900,1.24750,5936.399902,1.821325e+06,547.799988,1302.525024,0.0,273.725006,246.350006,793997.062500,65.675003,364995.531250,87.525002,24.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1017.825012,0.0,0.0,1002.150024,1008.250000,0.0,0.000,54.400002,1014.375000,250.0,0.0,0.0,0.0,0.050,145.548752,31.918749,0.000,280.475006,100.000000,40078.300781,5.975,-4.500,3.925,0.000,C
3996,2023-04-14 10:00:00,4.725,1.25100,5246.450195,2.082458e+06,603.099976,1063.300049,0.0,273.250000,253.524994,899762.500000,40.049999,190313.218750,96.074997,24.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1018.250000,0.0,0.0,1002.575012,1008.700012,0.0,0.000,54.200001,1014.825012,250.0,0.0,0.0,0.0,0.100,162.998001,34.829498,0.000,280.000000,100.000000,39771.148438,5.900,-4.475,3.825,-0.025,C
3997,2023-04-14 11:00:00,4.825,1.25100,4615.250000,2.213842e+06,620.275024,790.400024,0.0,273.500000,229.850006,870119.187500,19.000000,106297.023438,98.074997,24.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1018.500000,0.0,0.0,1002.849976,1008.950012,0.0,0.000,54.949997,1015.099976,250.0,0.0,0.0,0.0,0.100,181.162247,35.753750,0.000,280.075012,99.949997,39710.953125,6.000,-4.700,3.750,-0.075,C
3998,2023-04-14 12:00:00,4

In [11]:
#normalizer = tf.keras.layers.Normalization()
#normalizer.adapt(trainsetX)

model = tf.keras.models.Sequential([
  tf.keras.layers.GaussianNoise(stddev=0.001, seed=42),
  tf.keras.layers.Dense(200, activation="tanh"),
  tf.keras.layers.Dense(200, activation="tanh"),
  tf.keras.layers.Dense(200, activation="tanh"),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dense(1, activation="relu")
])


In [12]:
model.compile(
  #optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_absolute_error'
  )

history = model.fit(trainsetX, 
                    trainsetY, 
                    batch_size=1000, 
                    epochs=2000,
                    validation_data=(evalsetX, evalsetY)
                    )

Epoch 1/2000
88/88 [==============================] - 3s 18ms/step - loss: 258.0978 - val_loss: 337.0426
Epoch 2/2000
88/88 [==============================] - 1s 16ms/step - loss: 229.8017 - val_loss: 307.8899
Epoch 3/2000
88/88 [==============================] - 1s 13ms/step - loss: 209.9907 - val_loss: 281.8597
Epoch 4/2000
88/88 [==============================] - 1s 13ms/step - loss: 198.2456 - val_loss: 277.6259
Epoch 5/2000
88/88 [==============================] - 1s 16ms/step - loss: 189.7548 - val_loss: 265.0587
Epoch 6/2000
88/88 [==============================] - 2s 22ms/step - loss: 181.3095 - val_loss: 249.4031
Epoch 7/2000
88/88 [==============================] - 2s 18ms/step - loss: 172.3610 - val_loss: 257.5521
Epoch 8/2000
88/88 [==============================] - 1s 16ms/step - loss: 165.2072 - val_loss: 231.3566
Epoch 9/2000
88/88 [==============================] - 1s 14ms/step - loss: 157.4460 - val_loss: 263.5321
Epoch 10/2000
88/88 [==============================] - 

KeyboardInterrupt: 

In [193]:
results = model.evaluate(evalsetX, evalsetY)

92/92 [==============================] - 0s 2ms/step - loss: 37.2858


In [308]:
preds = model.predict(evalsetX)
compare = pd.concat([evalsetY, pd.DataFrame(preds)], axis=1)
compare.to_csv("test1.csv")

161/161 [==============================] - 1s 3ms/step


In [241]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('A/X_test_estimated.parquet')

X_test_estimated_a = X_test_estimated_a.iloc[:,1:]
X_test_estimated_b = X_test_estimated_b.iloc[:,1:]
X_test_estimated_c = X_test_estimated_c.iloc[:,1:]
X_test_estimated_a = add_location(averageFeatures(removeNaN(X_test_estimated_a)), "A")
X_test_estimated_b = add_location(averageFeatures(removeNaN(X_test_estimated_b)), "B")
X_test_estimated_c = add_location(averageFeatures(removeNaN(X_test_estimated_c)), "C")


X_test_estimated = pd.concat([X_test_estimated_a, X_test_estimated_b,X_test_estimated_c]).rename(columns={"date_forecast" : "time"})


X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

X_test = pd.merge(X_test_estimated, X_test_targets, on=["time", "location"], how="right")

X_test = X_test.iloc[:,1:-3]
X_test = ((X_test-X_test.mean())/X_test.std()).fillna(value=0)

test_preds = pd.read_csv("sample_submission.csv")
test_preds["prediction"] = model.predict(X_test)
test_preds.to_csv("NN_normalized_relu_better.csv", index=False)

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_47899/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_47899/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="bfill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_47899/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_47899/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = f

68/68 [==============================] - 0s 2ms/step
